In [1]:
#!pip install transformers
import torch
from transformers import BertTokenizer,BertModel, BertConfig
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from evaluate import *
from process import *

In [3]:
vectorizer = CountVectorizer()
data = vectorizer.fit_transform("/data/brown.txt")
print(data)

ValueError: Iterable over raw text documents expected, string object received.

In [6]:
print(train)

TypeError: argument of type 'BertModel' is not iterable